Tali Zacks, Yuting Huang, Jennifer Shiyue Zhang

In [138]:
import pandas as pd
import matplotlib.pyplot as plt
import bqplot
import numpy as np
import ipywidgets
import traitlets


# Build a dashboard for the buildings data.

### Left component:
- Grid heat map
- Columns are County
- Rows are the governmental department (Agency Name) (note, the agency names might overlap with your plot -- there are ways around this or you can leave as is)
- Values are mean of total square footage for that set of criteria

### Right component:
- is a barplot/histogram
- x is the year
- y is total square footage acquired that year
- These two should be linked so that you can select cells and that will update the square footage plot.
- Things to think about:
  - Can you keep the x and y ranges static on the bar plot?
  - Can you change the style?

### In your write-up please address:
- Any data transformations or rescalings you did for your plot
- How you dealt with NaN's (empty entries) in your dataset
- Any aesthetic choices you made (colors, layout, plot size, label size) and why, and what you'd like to experiment with if you had more time.


In [139]:
buildings = pd.read_csv('https://uiuc-ischool-dataviz.github.io/is445_spring2022/week03/data/building_inventory.csv',
                       na_values = {"Square Footage":0,
                                    "Year Acquired":0,
                                    "Year Constructed":0})
buildings['agency code'] = buildings['Agency Name']
buildings

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3,agency code
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,1975.0,1975.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,NaN,NaN,432.0,1,0,0,Storage,NaN,NaN,Department of Transportation
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,NaN,NaN,330.0,1,0,0,Storage,NaN,NaN,Department of Transportation
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,NaN,1987.0,130.0,1,0,0,Storage,High Hazard,NaN,Department of Transportation
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,1971.0,1971.0,49552.0,1,1,0,Education,Education,Not provided,Illinois Community College Board


In [140]:
agencies = buildings['Agency Name'].unique()
agencies

array(['Department of Natural Resources', 'Department of Corrections',
       'Department of Human Services', 'Department of Transportation',
       'Department of State Police', 'Department of Military Affairs',
       'Department of Agriculture', 'Governors State University',
       'Department of Central Management Services',
       'Illinois State University', 'Historic Preservation Agency',
       'Department of Juvenile Justice', 'Southern Illinois University',
       'Illinois Medical District Commission', 'University of Illinois',
       "Department of Veterans' Affairs", 'Chicago State University',
       'Northern Illinois University', 'Office of the Secretary of State',
       'Illinois Emergency Management Agency',
       'Western Illinois University', 'Eastern Illinois University',
       'Northeastern Illinois University',
       'Illinois Community College Board',
       'Illinois Board of Higher Education',
       'IL State Board of Education', 'Department of Revenue',


In [141]:
#create acronyms so you can see the agency names on the heatmap but makes a list
def acronymize_list():
    acronyms = []
    for i in agencies:
        if 'Appellate' in i:
            if "Fifth" in i:
                i_num = i.replace("Fifth","5th")
                i_ac = "".join(i_num[0] for i_num in i_num.split())
                acronyms.append([i,i_ac])
            elif "Fourth" in i:
                i_num = i.replace("Fourth","4th")
                i_ac = "".join(i_num[0] for i_num in i_num.split())
                acronyms.append([i,i_ac])            
            elif "Second" in i:
                i_num = i.replace("Second","2nd")
                i_ac = "".join(i_num[0] for i_num in i_num.split())
                acronyms.append([i,i_ac])
            elif "Third" in i:
                i_num = i.replace("Third","3rd")
                i_ac = "".join(i_num[0] for i_num in i_num.split())
                acronyms.append([i,i_ac])
        elif "Northeastern" in i:
            acronyms.append(['Northeastern Illinois University','NEIU'])
        else:
            i_ac = "".join(i[0] for i in i.split())
            acronyms.append([i,i_ac])
    return acronyms

In [142]:
ac_list = acronymize_list()
ac_list

[['Department of Natural Resources', 'DoNR'],
 ['Department of Corrections', 'DoC'],
 ['Department of Human Services', 'DoHS'],
 ['Department of Transportation', 'DoT'],
 ['Department of State Police', 'DoSP'],
 ['Department of Military Affairs', 'DoMA'],
 ['Department of Agriculture', 'DoA'],
 ['Governors State University', 'GSU'],
 ['Department of Central Management Services', 'DoCMS'],
 ['Illinois State University', 'ISU'],
 ['Historic Preservation Agency', 'HPA'],
 ['Department of Juvenile Justice', 'DoJJ'],
 ['Southern Illinois University', 'SIU'],
 ['Illinois Medical District Commission', 'IMDC'],
 ['University of Illinois', 'UoI'],
 ["Department of Veterans' Affairs", 'DoVA'],
 ['Chicago State University', 'CSU'],
 ['Northern Illinois University', 'NIU'],
 ['Office of the Secretary of State', 'OotSoS'],
 ['Illinois Emergency Management Agency', 'IEMA'],
 ['Western Illinois University', 'WIU'],
 ['Eastern Illinois University', 'EIU'],
 ['Northeastern Illinois University', 'NEIU']

In [143]:
#replace agency code column (duplicate Agency Name) values with abbreviated agency codes
for i in buildings['Agency Name']:
    for a in ac_list:
        if i == a[0]:
            buildings['agency code']=buildings['agency code'].replace(i, a[1])

In [144]:
buildings

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3,agency code
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,1975.0,1975.0,144.0,1,1,0,Unusual,Unusual,Not provided,DoNR
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,DoNR
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,DoNR
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,DoNR
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,DoNR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,NaN,NaN,432.0,1,0,0,Storage,NaN,NaN,DoT
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,NaN,NaN,330.0,1,0,0,Storage,NaN,NaN,DoT
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,NaN,1987.0,130.0,1,0,0,Storage,High Hazard,NaN,DoT
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,1971.0,1971.0,49552.0,1,1,0,Education,Education,Not provided,ICCB


In [145]:
# table = pd.pivot_table(buildings, values = 'Square Footage', index = ['Agency Name'],
#                        columns = ['County'], aggfunc=np.mean)
table = pd.pivot_table(buildings, values = 'Square Footage', index = ['Agency Name','agency code'],
                       columns = ['County'], aggfunc=np.mean)
table.fillna(0)
#Index == row, columns == columns

,County,Adams,Alexander,Bond,Boone,Brown,Bureau,Calhoun,Carroll,Cass,Champaign,...,Wabash,Warren,Washington,Wayne,White,Whiteside,Will,Williamson,Winnebago,Woodford
Agency Name,agency code,,,,,,,,,,,,,,,,,,,,,
Appellate Court / Fifth District,AC/5D,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Appellate Court / Fourth District,AC/4D,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Appellate Court / Second District,AC/2D,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Appellate Court / Third District,AC/3D,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chicago State University,CSU,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Department of Agriculture,DoA,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Department of Central Management Services,DoCMS,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,32857.500000,...,0.000000,10500.000000,0.000000,9200.000000,0.000000,0.000000,0.000000,42280.000000,70731.500000,0.000000
Department of Corrections,DoC,8613.333333,16422.863636,0.00,0.0,16046.307692,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15106.622093,0.000000,0.000000,0.000000
Department of Human Services,DoHS,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16668.250000,0.000000,20653.250000,0.000000


In [146]:
counties = table.columns.to_numpy()
agencies = table.index.to_numpy()

In [147]:
counties

array(['Adams', 'Alexander', 'Bond', 'Boone', 'Brown', 'Bureau',
       'Calhoun', 'Carroll', 'Cass', 'Champaign', 'Christian', 'Clark',
       'Clay', 'Clinton', 'Coles', 'Cook', 'Crawford', 'Cumberland',
       'DeKalb', 'DeWitt', 'Douglas', 'DuPage', 'Edgar', 'Effingham',
       'Fayette', 'Ford', 'Franklin', 'Fulton', 'Gallatin', 'Greene',
       'Grundy', 'Hamilton', 'Hancock', 'Hardin', 'Henderson', 'Henry',
       'Iroquois', 'Jackson', 'Jasper', 'Jefferson', 'Jersey',
       'Jo Daviess', 'Johnson', 'Kane', 'Kankakee', 'Kendall', 'Knox',
       'LaSalle', 'Lake', 'Lawrence', 'Lee', 'Livingston', 'Logan',
       'Macon', 'Macoupin', 'Madison', 'Marion', 'Marshall', 'Mason',
       'Massac', 'McDonough', 'McHenry', 'McLean', 'Menard', 'Mercer',
       'Monroe', 'Montgomery', 'Morgan', 'Moultrie', 'Ogle', 'Peoria',
       'Perry', 'Piatt', 'Pike', 'Pope', 'Pulaski', 'Putnam', 'Randolph',
       'Richland', 'Rock Island', 'Saline', 'Sangamon', 'Schuyler',
       'Scott', 'Shelby', 

In [148]:
agencies
agency = [i[0] for i in agencies]
agency_code = [i[1] for i in agencies]

In [149]:
##GRID HEAT MAP##

# 2. scales
col_sc = bqplot.ColorScale(scheme='Greens')
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# 3. axis
col_ax = bqplot.ColorAxis(scale=col_sc, orientation='horizontal', side='top', label = 'log10 Color Scale: ')
x_ax = bqplot.Axis(scale=x_sc, label='County',tick_rotate=-90, tick_style ={'font-size':'8px'},  label_offset ='50px')
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical', label='Agency Name',tick_offset = '50px',tick_style ={'font-size':'8px'}, label_offset ='50px')

# 4. marks
heat_map = bqplot.GridHeatMap(color = np.log10(table.values),
                             row = agency_code, #table 2 has acronym names
                             column = counties, 
                             scales = {'color':col_sc,'row':y_sc, 'column':x_sc},
                             interactions={'click':'select'},
                             selected_style={'fill':'pink'})

fig = bqplot.Figure(marks=[heat_map], axes=[col_ax,x_ax, y_ax])


In [150]:
fig

Figure(axes=[ColorAxis(label='log10 Color Scale: ', scale=ColorScale(scheme='Greens'), side='top'), Axis(label…

In [151]:
myLabel = ipywidgets.Label()

In [152]:
def on_selection(change):
    if len(change['owner'].selected==1): # only allow user to select one grid
        i,j = change['owner'].selected[0]
        v = table.values[i,j]
        myLabel.value = 'Mean of Total Square Footage for ' + agency[i] + ' in ' + counties[j] + ' County : '+ str(v) + ' ft^2'
        
        


In [153]:
heat_map.observe(on_selection,'selected')

In [154]:
fig.layout.min_width='300px'
fig.layout.min_height='300px'
myDashboard = ipywidgets.VBox([myLabel,fig])

In [155]:
myDashboard

## Right component

In [156]:
buildings = pd.read_csv('https://uiuc-ischool-dataviz.github.io/is445_spring2022/week03/data/building_inventory.csv',
                       na_values = {"Square Footage":0,
                                    "Year Acquired":0,
                                    "Year Constructed":0})
buildings['agency code'] = buildings['Agency Name']
buildings

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3,agency code
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,1975.0,1975.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,NaN,NaN,432.0,1,0,0,Storage,NaN,NaN,Department of Transportation
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,NaN,NaN,330.0,1,0,0,Storage,NaN,NaN,Department of Transportation
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,NaN,1987.0,130.0,1,0,0,Storage,High Hazard,NaN,Department of Transportation
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,1971.0,1971.0,49552.0,1,1,0,Education,Education,Not provided,Illinois Community College Board


In [157]:
buildings['Square Footage'].min()

9.0

In [158]:
buildings['Square Footage'].max()

1200000.0

In [159]:
buildings["Year Acquired"].min(), buildings["Year Acquired"].max()

(1753.0, 2019.0)

In [160]:
len(buildings['County'])

8862

In [161]:
len(buildings['Agency Name'])

8862

In [162]:
#we want to plot the amount of footage acquired per year

In [163]:
####### BAR PLOT ######
# for each year bin -- count up the total square footage
# 2. scales
x_scb = bqplot.LinearScale()
y_scb = bqplot.LinearScale()

# 3. axis 
x_axb = bqplot.Axis(label='Year Acquired', scale=x_scb)
y_axb = bqplot.Axis(label='Total Square Footage', scale=y_scb, orientation='vertical')

square_footage_hist = bqplot.Bars(x=buildings['Year Acquired'], y=buildings["Square Footage"], scales={'x':x_scb, 'y':y_scb})

In [164]:
fig_bars = bqplot.Figure(marks=[square_footage_hist],axes=[x_axb, y_axb])
fig_bars

Figure(axes=[Axis(label='Year Acquired', scale=LinearScale(), side='bottom'), Axis(label='Total Square Footage…

In [165]:
myLabel = ipywidgets.Label()

In [166]:
def on_selection(change):
    if len(change['owner'].selected==1): # only allow user to select one grid
        i,j = change['owner'].selected[0]
        v = table.values[i,j]
        myLabel.value = 'Mean of Total Square Footage for ' + agency[i] + ' in ' + counties[j] + ' County : '+ str(v) + ' ft^2'
        
heat_map.observe(on_selection,'selected')





In [167]:
figures = ipywidgets.HBox([fig,fig_bars])
fig.layout.min_width='550px'
fig_bars.layout.min_width='550px'
dashboard = ipywidgets.VBox([myLabel,figures])

In [168]:
#put two plots together, but haven't finished the interactive feature
dashboard

### build the link between the heat map and the bar chart

In [169]:
buildings = pd.read_csv('https://uiuc-ischool-dataviz.github.io/is445_spring2022/week03/data/building_inventory.csv',
                       na_values = {"Square Footage":0,
                                    "Year Acquired":0,
                                    "Year Constructed":0})
buildings['agency code'] = buildings['Agency Name']
buildings

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3,agency code
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,1975.0,1975.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided,Department of Natural Resources
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,NaN,NaN,432.0,1,0,0,Storage,NaN,NaN,Department of Transportation
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,NaN,NaN,330.0,1,0,0,Storage,NaN,NaN,Department of Transportation
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,NaN,1987.0,130.0,1,0,0,Storage,High Hazard,NaN,Department of Transportation
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,1971.0,1971.0,49552.0,1,1,0,Education,Education,Not provided,Illinois Community College Board


In [170]:
####### BAR PLOT ######
# for each year bin -- count up the total square footage
# 2. scales
x_scb = bqplot.LinearScale()
y_scb = bqplot.LinearScale()
#reference:https://github.com/bqplot/bqplot/issues/608
col_sc = bqplot.OrdinalColorScale(colors=['Orange', 'Purple'])

# 3. axis 
x_axb = bqplot.Axis(label='Year Acquired', scale=x_scb)
y_axb = bqplot.Axis(label='Total Square Footage', scale=y_scb, orientation='vertical')


# 4. moving the label perpendicular to the y axis
#reference:https://notebook.community/rmenegaux/bqplot/examples/Axis%20Properties
y_axb.label_offset = '6.5ex'

#bplt.xlim(1753, 2019)
#bplt.ylim(9, 1200000)



In [171]:
square_footage_hist = bqplot.Bars(x=buildings['Year Acquired'], y=buildings["Square Footage"], scales={'x':x_scb, 'y':y_scb, 'color':col_sc})

In [172]:
#I cannot understand how python choose those colors for bar. 
#If I delete the square_footage_hist.color or I only type square_footage_hist.color with no arguments, the plot would be the original blue color.
square_footage_hist.color = buildings["Year Acquired"].replace('1800', 'Orange').replace('2000', 'Purple')

In [173]:
fig_bars = bqplot.Figure(marks=[square_footage_hist],axes=[x_axb, y_axb])
fig_bars

Figure(axes=[Axis(label='Year Acquired', scale=LinearScale()), Axis(label='Total Square Footage', label_offset…

In [174]:
myyLabel = ipywidgets.Label()

In [175]:
square_footage_hist.x

array([1975., 2004., 2004., ...,   nan, 1971.,   nan])

In [176]:
square_footage_hist.y

array([  144.,   144.,   144., ...,   130., 49552.,   288.])

In [177]:
#Square_footage_hist.x
#Use the loc function to find the Year Acquired by county which I chose from the heatmap
#Square_footage_hist.y
#Use the loc function to find the Square Footage by county which I chose from the heatmap
def on_selection1(change):
    if len(change['owner'].selected==1): # only allow user to select one grid
        i,j = change['owner'].selected[0]
        v = table.values[i,j]
        myyLabel.value = 'Mean of Total Square Footage for ' + agency[i] + ' in ' + counties[j] + ' County : '+ str(v) + ' ft^2'
        square_x = buildings.loc[buildings["County"]==counties[j]]
        square_footage_hist.x = square_x["Year Acquired"]
        square_y = buildings.loc[buildings["County"]==counties[j]]
        square_footage_hist.y = square_y["Square Footage"]
        
        
heat_map.observe(on_selection1,'selected')




In [178]:
figures = ipywidgets.HBox([fig,fig_bars])
fig.layout.min_width='550px'
fig_bars.layout.min_width='550px'
dashboard = ipywidgets.VBox([myyLabel,figures])

In [179]:
dashboard

### Things to think about:

In [180]:
#1. Can you keep the x and y ranges static on the bar plot?
#No, I can't. Because the x axis and y axis in the bar chart are decided by the County. 
#I tried to use the bqplot.pyplot to set the xlim and ylim, but the error comes out: Set the domain bounds of the current 'x' scale.

#2. Can you change the style?
#I use color scale for color style, but I tried to use bqplot.marks.Bars.color to change the bar color. 
#However, I can not use the dropdown ipywidget function to select the color because bqplot.marks.Bars.color is a list

### Write Up

#### Any data transformations or rescalings you did for your plot？

In the left component, first, we created a new column “Agency code” to indicate agency name. To display governmental departments in plot in an easier way, we used for loops to create acronyms for departments. For example, “AC/5D” was short for “Appellate Court / Fifth District.” “UoI” was short for “University of Illinois.” Besides, in the created table, we calculated the mean of square footage of each agency as grids shown in the left component.

In the right component, we summed up square footage acquired of each agency as y-axis.

#### How you dealt with NaN's (empty entries) in your dataset？

We decided to change NaN values into zero using .fillna() function.

#### Any aesthetic choices you made (colors, layout, plot size, label size) and why, and what you'd like to experiment with if you had more time.

We made many aesthetic choices. For instance:

We set grid color of the heat map as green and selected grid as pink.
We set legend of the heat map to the top and display in horizontal.
Each grid in the heat map was 300px x 300px.
We displayed agency acronyms in x-axis in vertical for saving more spaces. Font sizes for all labels were 8px and were 50px away from grids.

The bin width of the bar chart was 550px.
For the bin color, we use orange and purple to represent. We try to make those two colors are meaningful, then we decide the bin on the 1800 year acquired should be orange, and the bin on the 2000 year acquired should be purple. However, it does not work.
If we have more time, perhaps we can try to find a method to solve the color problem or highlight (i.e., display in another color) the year with the most square footage of each agency in the right bar chart.

## My write up-- Yuting Huang

### what you learned and contributed to the submission?

I made the right component of the assignment. First, I built a bar chart, the x-axis is Year Acquired, and the y-axis is Total Square Footage. Next, I tried to connect the heat map and bar chart, but it did not work. Therefore, I found that I have to change the on_selection function. When I clicked the heat map, I was able to know the county name that would helps to find the year acquired, and the square footage. 

The questions I met:
I tried to change the colormap for adding more features in my bar plot, but I did not find the color map support bqplot. Furthermore, I gave a bar plot color scale and applied orange and purple to represent the 1800 year acquired and the 2000 year acquired. However, It did not work. I could find out the solution if I had more time.

### what you learned from others and their contributions to the submission?
I learn many useful methods from Tali. First, she utilized the acronymize_list to find the Agency Name’s abbreviation which is built for agency code. That is excellent work. The work helps the heat map look clear and easy_to_read. Then, she built the PD.pivot_table to organize County, Agency Name, and agency code variables and transform them into the NumPy array. I think Tali’s overall work is a hard code, and the heat map looks pretty. I also love how Jennifer organizes our work into write-up session